## REACT

In [1]:
!pip install chromadb
!pip install --upgrade chromadb
!pip install -U langchain-chroma
!pip install --upgrade openai langchain
!pip install --upgrade langchain chromadb
!pip install -U langchain-community
!pip install pysqlite3-binary
!pip install sentry-sdk==1.39.1
!pip install tqdm
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_chroma-0.2.5-py3-none-any.whl.metadata (1.1 kB)
  Using cached langchain_core-0.3.74-py3-none-any.whl.metadata (5.8 kB)
  Using cached langsmith-0.4.16-py3-none-any.whl.metadata (14 kB)
Using cached langchain_core-0.3.74-py3-none-any.whl (443 kB)
Using cached langsmith-0.4.16-py3-none-any.whl (375 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.142
    Uninstalling langsmith-0.1.142:
      Successfully uninstalled langsmith-0.1.142
  Attempting uninstall: 

In [2]:
!pwd

/project/work/flow/4.


In [1]:
import os
os.environ["CHROMA_TELEMETRY"] = "False"
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
import chromadb
from tqdm import tqdm

os.environ['OPENAI_API_VERSION'] = '2024-02-15-preview'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://upappliance.openai.azure.com'
os.environ['AZURE_OPENAI_API_KEY'] = '4gWgDnxVjVaIRQhPJFn2seBhFGhsgfZhpedg6J0lmWDdsltOAlD8JQQJ99AKACYeBjFXJ3w3AAABACOGylYc'
os.environ['AZURE_OPENAI_API_DEPLOYMENT_NAME'] = "gpt-4o-mini"  # 또는 "up_gpt4o"

In [ ]:
from tqdm import tqdm
import pandas as pd
import json
import os
from openai import AzureOpenAI
from new_keyword_pipeline import (
    setup_vectordb,
    format_top10_for_keyword_prompt,
    generate_new_keyword
)
from react_converted import main as sim_prompt_main

# ───────────────── 1. OpenAI 클라이언트 & 벡터 DB ──────────────────
client = AzureOpenAI(
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
    api_key=os.environ['AZURE_OPENAI_API_KEY'],
    api_version=os.environ['OPENAI_API_VERSION']
)
# vectordb2 = setup_vectordb("./keyword_list.csv")

# ───────────────── 2. 테스트 셋 로드 (필요시 행 수 조절) ───────────────
# test_df = pd.read_csv(
#     "./testset_newdata_200.csv",
#     encoding="utf-8-sig"
# ).iloc[100:200]

test_df = pd.read_csv('/datasets/DTS2025000173/data/test_sample_icc_group.csv', encoding='utf-8-sig')

# datasets/DTS2025000171/data/test_sample_new_100_group.csv
# datasets/DTS2025000172/data/test_sample_duplicate_100_group.csv
# /datasets/DTS2025000173/data/test_sample_icc_group.csv
results = []                                # 최종 결과 저장용 리스트
all_icc_list = []

# ───────────────── 3. 메인 루프 ─────────────────────────────────────
for idx, row in tqdm(test_df.iterrows(),
                     total=len(test_df),
                     desc="Processing Rows"):

    # 3-1. 지역 구분에 따라 LLM 입력 텍스트 선택
    if row["지역"] == "KR":
        input_text_for_chain = row["afterchange"]
    elif row["지역"] == "US":
        input_text_for_chain = row["beforechange"]
    else:
        print(f"[Row {idx}] Warning: 알 수 없는 '지역' 값 → afterchange 사용")
        input_text_for_chain = row["afterchange"]

    # 3-2. sim_prompt_main 에게 넘길 딕셔너리 구성
    input_dict_for_sim_prompt = {
        "component_group":   row["component_group"],
        "components":        row["components"],
        "original_input":    input_text_for_chain,
        "ticket_id_hashed":  row.get("ticket_id_hashed", ""),
        "지역":               row.get("지역", ""),
        "keyword":           row.get("keyword", "")
    }

    # 3-3. 파이프라인 실행 및 예외 처리
    try:
        # ★ 언패킹에 SC 원문 리스트 추가 (변경점 #1)
        (ticket_id, components, before_change, after_change,
         generated_summary, sim_score, sim_keyword,
         timing_results, proposal_summary_all, component_group,
         final_result_raw, token_usage_results, icc_part,
         top_10_table, top_10_records, sc_raw_list) = sim_prompt_main(input_dict_for_sim_prompt)

        # ICC 누적 저장
        if icc_part is not None and not icc_part.empty:
            all_icc_list.append(icc_part.copy())

        # ★ sim_score가 0이면 Top-10 1순위로 보정 (변경점 #2)
        if (sim_score is None or sim_score == 0) and top_10_records:
            try:
                sim_score = round(top_10_records[0]["similarity"] * 100, 2)
            except Exception:
                sim_score = 0.0

        # ★ sim_keyword 비었으면 현재 행 keyword로 채움 (변경점 #3)
        if not sim_keyword:
            sim_keyword = row.get("keyword", "")

        # ★ ticket_id가 비었을 수 있으므로 안전한 id 준비 (변경점 #4)
        ticket_id_for_kw = ticket_id or row.get("ticket_id_hashed", "")

        # 3-4. 키워드 후속 로직
        if sim_score >= 90:
            new_keyword = sim_keyword or row.get("keyword", "")
            print(f"[{ticket_id_for_kw}] 유사도 90↑ 기존 키워드 유지: {new_keyword}")
        else:
            gen_kw_result = generate_new_keyword(
                client=client,
                ticket_id=ticket_id_for_kw,                   # 보강 id 사용
                component_group=component_group,
                components=components,
                beforechange=before_change,
                afterchange=after_change,
                gen_sum=generated_summary,
                sim_score=sim_score,
                sim_keyword=sim_keyword,
                top10_records=top_10_records,                 # 필요시 테이블 MD도 추가 가능
            )
            new_keyword = gen_kw_result.get("new_keyword", sim_keyword or row.get("keyword", ""))
            print(f"[{ticket_id_for_kw}] 유사도 90↓ 새 키워드 생성: {new_keyword}")

        # ★ SC 원문을 그대로 보관 (JSON 문자열로) (변경점 #5)
        sc_raw_json = json.dumps(sc_raw_list, ensure_ascii=False) if sc_raw_list is not None else "[]"

        # 3-5. 결과 리스트에 저장
        results.append({
            "ticket_id_hashed":   row['ticket_id_hashed'],
            "component_group":    row["component_group"],
            "components":         row["components"],
            "beforechange":       row["beforechange"],
            "afterchange":        row["afterchange"],
            "지역":               row["지역"],
            "generated_summary":  generated_summary,
            "sim_score":          sim_score,
            "sim_keyword":        sim_keyword,
            "final_keyword":      new_keyword,
            "final_result_raw":   final_result_raw,          # 최종 JSON 원문 그대로
            "sc_raw_json":        sc_raw_json,               # SC 3회 원문 리스트
            "timing_results":     timing_results,
            "token_usage_results": token_usage_results,
            "proposal_summary_all": proposal_summary_all,     # 이 값도 LLM 원문(문자열) 상태일 수 있음
            "top10_table_md":     top_10_table,              # 마크다운 표 원문
            # "top10_records_json": json.dumps(top_10_records, ensure_ascii=False),  # 필요하면 주석 해제
        })

    except Exception as e:
        print(f"[Row {idx}] 처리 중 오류: {e}")
        continue

# ───────────────── 4. 결과 CSV 저장 ──────────────────────────────────
out_path = "result_icc_new_1.csv"
pd.DataFrame(results).to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"✅ 전체 완료! {out_path} 로 저장되었습니다.")

if all_icc_list:
    icc_df_final = pd.concat(all_icc_list, ignore_index=True).drop_duplicates()
    icc_df_final.to_csv("./icc_icc_1.csv", index=False, encoding="utf-8-sig")
    print("🗂️ ICC 데이터도 icc_icc_1.csv 로 저장했습니다.")


## COT

In [ ]:
# import pandas as pd
# import os
# from openai import OpenAI
# from new_keyword_pipeline import setup_vectordb, search_top10_keywords, generate_new_keyword
# from cot import main as sim_prompt_main

# os.environ["OPENAI_API_KEY"] = "sk-"
# client = OpenAI()
# df = pd.read_csv('./all_origin_updated.csv', encoding='utf-8-sig')

# keyword_vectordb = setup_vectordb("./keyword_list.csv")

# input = {
#     "components": "로봇청소기",
#     "generated_summary": "ThinkQ 평면도상에 선을 그어 청소구역을 지정하도록 해주세요"
# }

# # sim_prompt_main에서 ICC/Proposal/유사도/키워드 추출
# result, icc_df = sim_prompt_main(df, input)

# ticket_id = result['ticket_id']
# components = result['components']
# beforechange = result['before_change']
# afterchange = result['after_change']
# gen_sum = result['generated_summary']
# ragas_score = result['ragas_score']
# sim_keyword = result['keyword']
    
# print(f"\n\n✅ [최종 결과]\n"
#       f"ticket_id: {ticket_id}\n"
#       f"components: {components}\n"
#       f"beforechange: {beforechange}\n"
#       f"afterchange: {afterchange}\n"
#       f"gen_sum: {gen_sum}\n"
#       f"RAGAS_score: {ragas_score}\n"
#       f"sim_keyword: {sim_keyword}")

# # Proposal 분기 및 처리
# if ragas_score >= 90:
#     print(f"[{ticket_id}] 유사도 90↑ 기존 키워드 업로드: {sim_keyword}")
# else:
#     top10_keywords = search_top10_keywords(keyword_vectordb, gen_sum, top_k=10)
#     result = generate_new_keyword(
#         client, ticket_id, components, beforechange, afterchange,
#         gen_sum, ragas_score, sim_keyword, top10_keywords
#     )
#     print(f"[{ticket_id}] 유사도 90↓ 새 키워드 생성: {result['new_keyword']}")

In [ ]:
flow/2_개발폴더_UP가전운영팀/phase2_hanyang/testset_inquiry_related_20250725_123340.csv